In [1]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [2]:
# Set the random seed for reproducibility
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [3]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('data.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

C:\Users\Dung\AppData\Local\Temp\ipykernel_4580\681518283.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('data.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
C:\Users\Dung\AppData\Local\Temp\ipykernel_4580\681518283.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('data.csv', index_col="date", infer_datetime_format=True, parse_dates=True)


,fng_value
date,
2024-11-26,79
2024-11-25,82
2024-11-24,80
2024-11-23,93
2024-11-22,94


In [4]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('ETH_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

C:\Users\Dung\AppData\Local\Temp\ipykernel_4580\3906409683.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df2 = pd.read_csv('ETH_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']


Date
31-08-2023    1645.639160
31-08-2024    2513.393799
31-10-2023    1816.458984
31-10-2024    2515.799316
31-12-2023    2281.471191
Name: Close, dtype: float64

In [5]:
print(df.dtypes)

fng_value    int64
dtype: object


In [6]:
common_dates = df.index.intersection(df2.index)
print("Common dates:", common_dates)

Common dates: Index([], dtype='object')


In [7]:
df.index = pd.to_datetime(df.index, dayfirst=True)
df2.index = pd.to_datetime(df2.index, dayfirst=True)

In [8]:
print(df.dtypes)

fng_value    int64
dtype: object


In [9]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2023-01-05,29,1250.438599
2023-01-04,29,1256.526611
2023-01-03,26,1214.778809
2023-01-02,27,1214.656616
2023-01-01,26,1200.964844


In [10]:
df.head()

,fng_value,Close
2024-11-26,79,3326.517334
2024-11-25,82,3413.543945
2024-11-24,80,3363.659912
2024-11-23,93,3396.223389
2024-11-22,94,3331.600830


In [11]:
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [12]:
# Predict Closing Prices using a 1 day window of previous closing prices
window_size = 1

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [30]:
# Use 70% of the data for training and the remaineder for testing
split = int(0.7 * len(X))
X_train = X[: split]
X_test = X
y_train = y[: split]
y_test = y

In [31]:
# Use the MinMaxScaler to scale data between 0 and 1.
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)

MinMaxScaler()

In [32]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler.fit(y)

y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [33]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [35]:
# Define the LSTM RNN model.
model = Sequential()

number_units = 50
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [36]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error")

In [37]:
# Summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 1, 50)             10400     
                                                                 
 dropout_3 (Dropout)         (None, 1, 50)             0         
                                                                 
 lstm_4 (LSTM)               (None, 1, 50)             20200     
                                                                 
 dropout_4 (Dropout)         (None, 1, 50)             0         
                                                                 
 lstm_5 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_5 (Dropout)         (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                

In [38]:
# Train the model
model.fit(X_train, y_train, epochs=15, shuffle=False, batch_size=1, verbose=1)

Epoch 1/15
485/485 [==============================] - 18s 24ms/step - loss: 0.0298
Epoch 2/15
485/485 [==============================] - 12s 25ms/step - loss: 0.0115
Epoch 3/15
485/485 [==============================] - 13s 26ms/step - loss: 0.0120
Epoch 4/15
485/485 [==============================] - 10s 21ms/step - loss: 0.0116
Epoch 5/15
485/485 [==============================] - 9s 19ms/step - loss: 0.0103
Epoch 6/15
485/485 [==============================] - 10s 20ms/step - loss: 0.0095
Epoch 7/15
485/485 [==============================] - 11s 22ms/step - loss: 0.0082
Epoch 8/15
485/485 [==============================] - 13s 26ms/step - loss: 0.0068
Epoch 9/15
485/485 [==============================] - 13s 26ms/step - loss: 0.0061
Epoch 10/15
485/485 [==============================] - 12s 25ms/step - loss: 0.0050
Epoch 11/15
485/485 [==============================] - 12s 26ms/step - loss: 0.0049
Epoch 12/15
485/485 [==============================] - 11s 23ms/step - loss: 0.0040
Ep

## Model Performance

In [39]:
# Evaluate the model
model.evaluate(X_test, y_test)
len(X_test)

22/22 [==============================] - 2s 11ms/step - loss: 0.0145


693

In [40]:
# Make some predictions
predicted = model.predict(X_test)

22/22 [==============================] - 2s 9ms/step


In [41]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [42]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices):]) 
stocks.head()

,Real,Predicted
2024-11-24,3413.543945,2751.789062
2024-11-23,3363.659912,2791.682129
2024-11-22,3396.223389,2769.027344
2024-11-21,3331.600830,2783.880859
2024-11-20,3361.053955,2754.166992


In [43]:
stocks.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [44]:
# Predict the next day's price
# Extract the last window of data for prediction
last_window = df["Close"].iloc[-window_size:].values.reshape(-1, 1)
scaled_last_window = scaler.transform(last_window)

# Reshape the input to match the model's expected input shape
scaled_last_window = scaled_last_window.reshape((1, scaled_last_window.shape[0], 1))

# Make the prediction
next_day_prediction = model.predict(scaled_last_window)

# Inverse transform the predicted value to get the actual price
next_day_price = scaler.inverse_transform(next_day_prediction)

print(f"Predicted next day's closing price: {next_day_price[0][0]:.2f}")

1/1 [==============================] - 0s 39ms/step
Predicted next day's closing price: 1385.01
